In [ ]:
import numpy as np 
import pandas as pd 
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
sub_data=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
train_data.head()

In [ ]:
(train_data.isnull()==True).sum()

In [ ]:
train_data['target'].value_counts()

In [ ]:
train_data[train_data['keyword'].isnull()==False]

In [ ]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train_data[train_data['id'].isin(ids_with_target_error)]

In [ ]:
train_data.loc[train_data['id'].isin(ids_with_target_error),'target'] = 0
train_data[train_data['id'].isin(ids_with_target_error)]

In [ ]:
train_data.loc[train_data['keyword'].notnull(), 'text'] = train_data['keyword'] + ' ' + train_data['text']
# test.loc[test['keyword'].notnull(), 'text'] = test['keyword'] + ' ' + test['text']

# view
train_data[train_data['keyword'].notnull()].head()

# **Cleaning**

In [ ]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(strip_handles=True)
stop_words = set(stopwords.words('english'))
corpus = []

def clean_data(text):
    # special characters
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    # remove punctuation and special chars (keep '!')
    for p in string.punctuation.replace('!', ''):
        text = text.replace(p, '')
        
    # remove urls
    text = re.sub(r'http\S+', '', text)
    
    # tokenize
    text = tknzr.tokenize(text)
    
    # remove stopwords
    text = [w.lower() for w in text if not w in stop_words]
    corpus.append(text)
    
    # join back
    text = ' '.join(text)
    
    return text


   

In [ ]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
     "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
def convert_abbrev_in_text(text):
    t=[]
    words=text.split()
    t = [abbreviations[w.lower()] if w.lower() in abbreviations.keys() else w for w in words]
    return ' '.join(t)    

In [ ]:
train_data['text']=train_data['text'].apply(clean_data)
train_data['text']=train_data['text'].apply(convert_abbrev_in_text)

# **Tokenizing**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional,Dropout,BatchNormalization,GlobalMaxPool1D,Input
# from keras.layers import Embedding, Input, Dense, CuDNNGRU,Bidirectional, Dropout,SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# max_features = 50000
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
sequences = tokenizer.texts_to_sequences(train_data['text'])
pad=pad_sequences(sequences,maxlen=120,padding='post',truncating='post')

In [ ]:
partition=int(len(train_data)*0.8);
y_train=train_data['target'].iloc[0:partition]
y_val=train_data['target'].iloc[partition:len(train_data)]
x_train=pad[0:partition]
x_val=pad[partition:len(train_data)]

vocab_size=len(tokenizer.word_index)+1

# **Model Architecture**

In [ ]:
#after clear_data function is used 
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=pad.shape[1]))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(Dropout(0.7))

model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(Dropout(0.7))

model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.7))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(x_train, y_train, epochs=3, validation_data=(x_val, y_val))

In [ ]:
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_data.loc[test_data['keyword'].notnull(), 'text'] = test_data['keyword'] + ' ' + test_data['text']
test_data['text']=test_data['text'].apply(clean_data)
t_sequences = tokenizer.texts_to_sequences(test_data['text'])
t_pad=pad_sequences(t_sequences,maxlen=120,padding='post',truncating='post')

In [ ]:
y_pred=model.predict(t_pad)

In [ ]:
sub_file=pd.DataFrame()
sub_file['id']=test_data['id']
sub_file['target']=y_pred.round().astype(int)
sub_file.head()

In [ ]:
sub_file.to_csv('submission.csv', index=False)

# **Glove Embedding**

In [ ]:
import numpy as np
embedding_dict={}
with open('../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word = values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
all_embs = np.stack(embedding_dict.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]
word_index = tokenizer.word_index
nb_words = len(word_index)+1

embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

for word, i in word_index.items():
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
from keras.initializers import Constant
model_glove = Sequential()
model_glove.add(Embedding(nb_words,embed_size, input_length=pad.shape[1],weights= [embedding_matrix]))

model_glove.add(Bidirectional(LSTM(32,return_sequences=True)))
model_glove.add(Dropout(0.7))
model_glove.add(Bidirectional(LSTM(32,return_sequences=True)))
model_glove.add(Dropout(0.7))
model_glove.add(Bidirectional(LSTM(32)))
model_glove.add(Dropout(0.7))

model_glove.add(Dense(32, activation='tanh'))
model_glove.add(Dropout(0.7))

model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model_glove.fit(x_train, y_train, epochs=4, validation_data=(x_val, y_val))

In [ ]:
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_data.loc[test_data['keyword'].notnull(), 'text'] = test_data['keyword'] + ' ' + test_data['text']
test_data['text']=test_data['text'].apply(clean_data)
t_sequences = tokenizer.texts_to_sequences(test_data['text'])
t_pad=pad_sequences(t_sequences,maxlen=120,padding='post',truncating='post')

In [ ]:
y_glove_pred=model_glove.predict(t_pad)

In [ ]:
sub_glove_file=pd.DataFrame()
sub_glove_file['id']=test_data['id']
sub_glove_file['target']=y_glove_pred.round().astype(int)
sub_glove_file.head()

In [ ]:
sub_glove_file.to_csv("submission_glove.csv", index=False)

# **Download submission.csv file **

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe

# create a link to download the dataframe
create_download_link(sub_file)

In [ ]:
#before clear_data function is used 
# model = Sequential()
# model.add(Embedding(vocab_size, 64, input_length=pad.shape[1]))
# model.add(Bidirectional(LSTM(200, return_sequences=True)))
# model.add(LSTM(200))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# history = model.fit(x_train, y_train, epochs=1, validation_data=(x_val, y_val))

In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, 64, input_length=pad.shape[1]))
# model.add(Bidirectional(LSTM(64,return_sequences=True)))
# model.add(Bidirectional(LSTM(64)))
# model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.7))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# history = model.fit(x_train, y_train, epochs=1, validation_data=(x_val, y_val))

In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, 64, input_length=pad.shape[1]))
# model.add(Bidirectional(LSTM(64,return_sequences=True)))
# model.add(Bidirectional(LSTM(128)))
# model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.7))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])